In [1]:
# Imports for making dataframe
import pandas as pd
import numpy as np

In [2]:
# Import for crawling Goodreads
import requests

In [3]:
# Imports for XML
from xml.etree import ElementTree

In [4]:
# Import for finding texts in HTML
from bs4 import BeautifulSoup

In [5]:
# Imports for seeing full text reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
# Goodreads API Developer key
key = 'mlUZUnPildWfc5tlXU03w'
secret = 'vHaBborkweWsa1J2hB5IN6E4jDM7WlL3Xm0D2bcWAI'

In [7]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Goodreads login
email = ''
password = ''

In [8]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Load Amazon data to find reviews of same books from Goodreads
amazon = pd.read_csv('asinOnly3_8.csv')
books = amazon.asin
books

0      0061690279
1      0061690309
2      0061690317
3      006169035X
4      0061690368
          ...    
245    0061728519
246    006172856X
247    0061728616
248    0061728810
249    0061728837
Name: asin, Length: 250, dtype: object

In [9]:
"""### NOTE: COMMENT OUT FOR FULL SET
## TEST SUBSET
test = set()
for i,val in enumerate(books):
    if i == 3:
        break
    test.add(val)
books = test
books"""

'### NOTE: COMMENT OUT FOR FULL SET\n## TEST SUBSET\ntest = set()\nfor i,val in enumerate(books):\n    if i == 3:\n        break\n    test.add(val)\nbooks = test\nbooks'

In [10]:
# Create empty dataframe to store Goodreads reviews & ratings
column_names = ['overall', 'reviewTime', 'asin', 'reviewText']
df = pd.DataFrame(columns = column_names)
print(df)

Empty DataFrame
Columns: [overall, reviewTime, asin, reviewText]
Index: []


In [11]:
# Log in Goodreads
driver = webdriver.Chrome()
driver.get ('https://www.goodreads.com/user/sign_in?source=home')
driver.find_element_by_name('user[email]').send_keys(email)
driver.find_element_by_name('user[password]').send_keys(password)
driver.find_element_by_name('next').click()                            

In [12]:
# Get most popular reviews for each book
for book in books:
    
    # Try to look up the book in Goodreads. If a book isn't found it will not be added to the dataset -> DATA CLEANING LATER
    try:
        #print('TRYING BOOK:', book)
        
        # Search up book on Goodreads
        res = requests.get('https://www.goodreads.com/book/isbn', params={'format': 'xml', 'key': key, 'isbn': book})
        
        # Get XML from API
        root = ElementTree.fromstring(res.text)    
        
        # Find iframe from XML
        iframe = root.find('book').find('reviews_widget').text
        
        # Parse HTML of XML to find link to reviews
        soup = BeautifulSoup(iframe, 'html.parser')
        url = soup.find('iframe')['src']
        
        # Open link to reviews using Selenium
        driver.get(str(url))

        # PAGE 1: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
                            'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        # PAGE 2: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        driver.find_element_by_link_text('2').click()
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
                            'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()           
            driver.switch_to.window(driver.window_handles[0])
            
        # PAGE 3: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        driver.find_element_by_link_text('3').click()
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
                            'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()           
            driver.switch_to.window(driver.window_handles[0])
            
    except:
        pass
        #print('   NOT FOUND')

In [13]:
df

,overall,reviewTime,asin,reviewText
0,2,"Jun 25, 2011",0061690279,This is the funniest unintentionally funny boo...
1,2,"Jul 31, 2011",0061690279,I checked this book out of the library after m...
2,2,"Jun 07, 2011",0061690279,I was really intrigued by this book at the sta...
3,3,"Aug 20, 2011",0061690279,I am actually not sure who I would recommend t...
4,5,"May 07, 2012",0061690279,Although I didn't share Gilman's understanding...
...,...,...,...,...
6536,3,"Dec 23, 2012",0061728837,"Buku ini unik. Jalan ceritanya sih biasa aja, ..."
6537,1,"Mar 23, 2009",0061728837,"BELIEVE (Romance/TT) - NRAlexander, VictoriaFr..."
6538,1,"Jun 22, 2010",0061728837,Normally I sort of like Victoria Alexander's h...
6539,5,"May 18, 2011",0061728837,Tessa is a teacher who does not like the Middl...


In [14]:
## NOTE: CHANGE FILE NAME BEFORE RUNNING CODE
df.to_csv('grData3_8.csv',index=False)